In [1]:
import numpy as np
import pandas as pd

In [5]:
obs_x = [20.77773, 21.27308, 20.39091, 20.81981]
obs_y = [20.30254, 20.95612, 21.05057, 21.25931]
pred_x = [20.7645, 21.2761, 20.3933, 20.8199]
pred_y = [20.3032, 20.9608, 21.0459, 21.2593]
# pred_x = [20.7777, 21.2731, 20.3909, 20.8198]
# pred_y = [20.3025, 20.9561, 21.0506, 21.2593]
sigma = [0.00128, 0.00085, 0.00083, 0.00039]
# obs_x = np.round(obs_x, 5)
# obs_y = np.round(obs_y, 5)
# pred_x = np.round(pred_x, 5)
# pred_y = np.round(pred_y, 5)
# sigma = np.round(sigma, 5)

In [3]:
obs_x = [20.77773, 21.27308, 20.39091, 20.81981, 20.30254, 20.95612, 21.05057, 21.25931]
pred_x = [20.7645, 21.2761, 20.3933, 20.8199, 20.3032, 20.9608, 21.0459, 21.2593]
sigma = [0.00128, 0.00085, 0.00083, 0.00039, 0.00128, 0.00085, 0.00083, 0.00039]
chi2_x = np.sum(((np.array(obs_x) - np.array(pred_x)) / np.array(sigma)) ** 2)
print(f"Chi2 for x: {chi2_x}")

Chi2 for x: 190.03871090026337


In [ ]:
# chi2 calculation
chi2_x = np.sum(((np.array(obs_x) - np.array(pred_x)) / np.array(sigma)) ** 2)
chi2_y = np.sum(((np.array(obs_y) - np.array(pred_y)) / np.array(sigma)) ** 2)
chi2_total = chi2_x + chi2_y
print(f"Chi2 for x: {chi2_x}")
print(f"Chi2 for y: {chi2_y}")
print(f"Total Chi2: {chi2_total}")

Chi2 for x: 127.79987377380323
Chi2 for y: 62.23883712646014
Total Chi2: 190.03871090026337


In [11]:
import numpy as np

def chi2calc_point_iplane(obs_data, predicted_images, chi2_params):
    """
    Calculate image plane chi-squared for point sources
    
    Parameters:
    obs_data: array of shape (n_images, 7) containing:
        [x_obs, y_obs, flux_obs, pos_err, flux_err, td_obs, td_err]
    predicted_images: array of shape (n_pred, 4) containing:
        [x_pred, y_pred, magnification, time_delay]
    chi2_params: dict with keys:
        'chi2_usemag': 0 for flux, 1 for magnitude
        'chi2pen_nimg': penalty for wrong number of images
        'chi2pen_parity': penalty for parity violations
        'chi2pen_range': penalty for out of range parameters
    
    Returns:
    c2: array of length 5 [total, position, flux, time_delay, prior]
    """
    
    n_obs = len(obs_data)
    n_pred = len(predicted_images)
    
    # Initialize chi2 components
    c2 = np.zeros(5)  # [total, pos, flux, td, prior]
    
    if n_obs == 0:
        return c2
    
    # Check if we have the right number of predicted images
    if n_pred < n_obs:
        c2[0] = c2[1] = chi2_params['chi2pen_nimg']
        return c2
    
    # Image matching - find closest predicted image for each observation
    used_pred = np.zeros(n_pred, dtype=bool)
    matched_indices = np.zeros(n_obs, dtype=int)
    
    f1 = f2 = 0.0  # For flux normalization
    t1 = t2 = 0.0  # For time delay normalization
    
    for j in range(n_obs):
        min_dist2 = np.inf
        best_match = -1
        
        # Find closest unused predicted image
        for k in range(n_pred):
            if not used_pred[k]:
                # Calculate squared distance
                dx = predicted_images[k, 0] - obs_data[j, 0]
                dy = predicted_images[k, 1] - obs_data[j, 1]
                dist2 = dx*dx + dy*dy
                
                if dist2 < min_dist2:
                    min_dist2 = dist2
                    best_match = k
        
        if best_match == -1:
            c2[0] = c2[1] = chi2_params['chi2pen_nimg']
            return c2
        
        used_pred[best_match] = True
        matched_indices[j] = best_match
        
        # Position chi2 contribution
        if obs_data[j, 3] > 0.0:  # position error > 0
            c2[1] += min_dist2 / (obs_data[j, 3] * obs_data[j, 3])
        
        # Accumulate flux terms for normalization
        if obs_data[j, 4] > 0.0:  # flux error > 0
            mag_pred = predicted_images[best_match, 2]  # magnification
            if chi2_params['chi2_usemag'] == 0:
                # Use flux
                f1 += abs(obs_data[j, 2] * mag_pred) / (obs_data[j, 4] * obs_data[j, 4])
                f2 += (mag_pred * mag_pred) / (obs_data[j, 4] * obs_data[j, 4])
            else:
                # Use magnitude
                f1 += (obs_data[j, 2] + 2.5 * np.log10(abs(mag_pred))) / (obs_data[j, 4] * obs_data[j, 4])
                f2 += 1.0 / (obs_data[j, 4] * obs_data[j, 4])
        
        # Accumulate time delay terms for normalization
        if obs_data[j, 6] > 0.0:  # time delay error > 0
            td_pred = predicted_images[best_match, 3]  # time delay
            t1 += (obs_data[j, 5] - td_pred) / (obs_data[j, 6] * obs_data[j, 6])
            t2 += 1.0 / (obs_data[j, 6] * obs_data[j, 6])
    
    # Calculate flux and time delay normalizations
    flux_norm = f1 / f2 if f2 > 0.0 else 1.0
    td_norm = t1 / t2 if t2 > 0.0 else 0.0
    
    # Calculate flux and time delay chi2
    parity_violation = False
    
    for j in range(n_obs):
        k = matched_indices[j]
        mag_pred = predicted_images[k, 2]
        td_pred = predicted_images[k, 3]
        
        # Flux chi2
        if obs_data[j, 4] > 0.0:
            if chi2_params['chi2_usemag'] == 0:
                # Flux mode
                flux_diff = abs(obs_data[j, 2]) - abs(mag_pred) * flux_norm
                c2[2] += (flux_diff * flux_diff) / (obs_data[j, 4] * obs_data[j, 4])
            else:
                # Magnitude mode
                mag_diff = obs_data[j, 2] + 2.5 * np.log10(abs(mag_pred)) - flux_norm
                c2[2] += (mag_diff * mag_diff) / (obs_data[j, 4] * obs_data[j, 4])
        
        # Time delay chi2
        if obs_data[j, 6] > 0.0:
            td_diff = obs_data[j, 5] - td_pred - td_norm
            c2[3] += (td_diff * td_diff) / (obs_data[j, 6] * obs_data[j, 6])
    
    # Apply parity penalty if needed
    if parity_violation:
        c2[2] = chi2_params['chi2pen_parity']
    
    # Total chi2 (prior c2[4] would be added separately)
    c2[0] = c2[1] + c2[2] + c2[3] + c2[4]
    
    return c2

# Example usage:
def example_usage():
    # Example observation data: [x, y, flux, pos_err, flux_err, td, td_err]
    obs_data = np.array([
        [20.77773, 20.30254, 0.0, 0.00128, 0.0, 0.0, 0.0, 0],  # Image 1
        [21.27308, 20.95612, 0.0, 0.00085, 0.0, 0.0, 0.0, 0],
        [20.39091, 21.05057, 0.0, 0.00083, 0.0, 0.0, 0.0, 0],
        [20.81981, 21.25931, 0.0, 0.00039, 0.0, 0.0, 0.0, 0]  # Image 2
    ])
    
    # Example predicted images: [x, y, magnification, time_delay]
    predicted_images = np.array([
        [20.7645, 20.3032, -6.6948, 1.051],
        [21.2761, 20.9608, 13.6373, 0.000],
        [20.3933, 21.0459, 15.4906, 0.056],
        [20.8199, 21.2593, -19.6275, 0.169]
    ])
    
    # Chi2 parameters
    chi2_params = {
        'chi2_usemag': 0,
        'chi2pen_nimg': 1e30,
        'chi2pen_parity': 1e30,
        'chi2pen_range': 1e30
    }
    
    c2 = chi2calc_point_iplane(obs_data, predicted_images, chi2_params)
    print(f"Chi2 components: total={c2[0]:.6e}, pos={c2[1]:.6e}, flux={c2[2]:.6e}, td={c2[3]:.6e}, prior={c2[4]:.6e}")

In [12]:
example_usage()

Chi2 components: total=1.900387e+02, pos=1.900387e+02, flux=0.000000e+00, td=0.000000e+00, prior=0.000000e+00
